In [1]:
!pip install diffusers transformers torch

  Using cached diffusers-0.31.0-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached safetensors-0.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached diffusers-0.31.0-py3-none-any.whl (2.9 MB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached huggingface_hub-0.26.5-py3-none-any.whl (447 kB)
Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (792 kB)
Using cached safetensors-0.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 M

In [2]:
# HARD CODED TO DELETE
prompt = 'contrary subject hearing small law'
#negative_prompt = 'blurry, text, letters, typography, writing, low resolution, watermark, patterns, distorted face, bad anatomy, uncanny, extra fingers, missing fingers, missing faces, pixelated, deformed'
negative_prompt = 'blurry, text, letters, typography, writing, low resolution, watermark, patterns, distorted face, bad anatomy, uncanny, extra fingers, missing fingers, missing faces, pixelated, deformed, no text'
positive_prompt = 'clean lines, well-balanced composition'

In [3]:
words = prompt.split()
height = 1080/len(words)  # Set height
width = 1080  # Set width

In [4]:
import random
from io import BytesIO

import requests
import torch
from PIL import Image

from diffusers import (
    AutoPipelineForImage2Image,
    ControlNetModel,
    DPMSolverMultistepScheduler,
    StableDiffusionXLControlNetPipeline,
)
from diffusers.image_processor import IPAdapterMaskProcessor
from diffusers.utils import load_image, logging
from diffusers.utils.logging import set_verbosity

In [5]:
set_verbosity(logging.ERROR)

In [6]:
from diffusers import ControlNetModel
import torch

controlnet = ControlNetModel.from_pretrained(
    "destitech/controlnet-inpaint-dreamer-sdxl", torch_dtype=torch.float16, variant="fp16"
)

In [9]:
pipeline = StableDiffusionXLControlNetPipeline.from_pretrained("RunDiffusion/Juggernaut-XL-v9",
                                                               torch_dtype=torch.float16,
                                                               variant="fp16",
                                                               controlnet=controlnet).to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [ ]:
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
pipeline.scheduler.config.use_karras_sigmas = True

In [ ]:
pipeline.load_ip_adapter(
    "h94/IP-Adapter",
    subfolder="sdxl_models",
    weight_name="ip-adapter-plus_sdxl_vit-h.safetensors",
    image_encoder_folder="models/image_encoder")

In [ ]:
pipeline.set_ip_adapter_scale(0.4)

In [ ]:
source_image = Image.open("images_generated_to_clean2/expanded_only.jpg").convert("RGB")
source_image

In [ ]:
canvas_width, canvas_height = 1024, 512
blank_canvas = Image.new("RGBA", (canvas_width, canvas_height), "WHITE")

In [ ]:
x_offset = (canvas_width - source_image.width) // 4
y_offset = (canvas_height - source_image.height) // 4
print(f'x={x_offset}, y={y_offset}')
blank_canvas.paste(source_image, (x_offset, y_offset))
blank_canvas

In [ ]:
mask = Image.new("L", (canvas_width, canvas_height), 0) # L for luminance --> grayscale mode                 
mask.paste(255, (x_offset, y_offset, x_offset + source_image.width, y_offset + source_image.height))
mask

In [ ]:
processor = IPAdapterMaskProcessor()
ip_masks = processor.preprocess(mask, height=canvas_height, width=canvas_width)

In [ ]:
ip_masks

In [ ]:
seed = 13
generator = torch.Generator(device="cpu").manual_seed(seed)

In [ ]:
latents = pipeline(prompt=prompt + positive_prompt,
                   height=canvas_height,
                   width=canvas_width,
                   guidance_scale=6.5,
                   num_inference_steps=25,
                   generator=generator,
                   image=mask,
                   controlnet_conditioning_scale=0.9,
                   control_guidance_end=0.9,
                   ip_adapter_image=blank_canvas,
                   cross_attention_kwargs={"ip_adapter_masks": ip_masks},
                   output_type="latent").images[0]

In [ ]:
latents

In [ ]:
pipeline_img2img = AutoPipelineForImage2Image.from_pipe(pipeline, controlnet=None)

In [ ]:
cadavre_exquis01 = pipeline_img2img(prompt=prompt+positive_prompt,
                                    negative_prompt=negative_prompt,
                                    guidance_scale=3.0,
                                    num_inferences_steps=25,
                                    generator=generator,
                                    image=latents,
                                    strength=0.2,
                                    ip_adapter_image=blank_canvas,
                                    cross_attention_kwargs={"ip_adapter_masks": ip_masks}).images[0]


In [ ]:
cadavre_exquis01.save("outpainting_exquis_01.png")

# UTILISER https://huggingface.co/docs/diffusers/en/advanced_inference/outpaint
https://huggingface.co/destitech/controlnet-inpaint-dreamer-sdxl

https://github.com/huggingface/diffusers/discussions/7482

https://huggingface.co/destitech/controlnet-inpaint-dreamer-sdxl

Regénérer des plus longues prompts, éventuellement que des noms, etc.

# Reset

In [51]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()
%reset -f



In [26]:
!nvidia-smi

Sat Dec  7 22:07:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A16                     On  | 00000000:27:00.0 Off |                    0 |
|  0%   41C    P0              28W /  62W |  13918MiB / 15356MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--